In [51]:
import os
import pandas as pd
import os
from soundtype_params import SoundtypeParams
import numpy as np
import xlrd
import pandas as pd

# F:\中建科技项目数据\history\use\2021-12-27\xls\Record109848.xls

root_dir = r"F:\中建科技项目数据\history\use"
file_name = "Record109848" + ".xls"

In [52]:
# 输入根目录和文件名，返回文件路径
def seek_files(root_dir, file_name):
    for root, dirs, files in os.walk(root_dir):
        if file_name in files:
            path = "{0}\\{1}".format(root, file_name)
            path = path.replace('\\', '/')
    return path

In [43]:
path = seek_files(root_dir, file_name)
path

'F:/中建科技项目数据/history/use/2021-12-27/xls/Record109848.xls'

In [65]:
# 读取麒麟山庄的对照表
qilin_path = r'.\input\2021 新闻大厦 麒麟山庄数据.xlsx'
qilin_name = "麒麟山庄"
qilin_data = pd.read_excel(qilin_path, sheet_name=qilin_name)
# 抽取出leq在50-70之间的
qilin_use_data = qilin_data[(float(50) < qilin_data['Leq']) & (qilin_data['Leq'] < float(70))]
# 重新编号
qilin_use_data.reset_index(drop=True, inplace=True)
qilin_use_data

,id,file_name,place,create_time,Leq,Pctn,ANL
0,2,Record85823,M01,2021-11-06 11:36:47,54.3125,0.146498,3
1,4,Record85826,M01,2021-11-06 11:42:46,55.2321,0.205802,3
2,6,Record85828,M01,2021-11-06 11:46:46,50.5334,0.361827,3
3,66,Record85897,M01,2021-11-06 14:04:43,50.2061,0.406511,3
4,119,Record85955,M01,2021-11-06 16:00:43,50.8922,0.410303,3
...,...,...,...,...,...,...,...
751,17032,Record109726,M01,2021-12-27 05:58:38,50.0249,0.154973,3
752,17061,Record109758,M01,2021-12-27 08:06:43,57.3701,0.227073,3
753,17086,Record109784,M01,2021-12-27 09:50:46,51.3272,0.266403,3
754,17103,Record109802,M01,2021-12-27 11:02:49,55.5277,0.082539,3


In [54]:
# 提取每个文件的特征
def extract_features(path):
    wb = xlrd.open_workbook(path)
    params_dict = {}

    for idx in range(6):
        sheet = wb.sheet_by_index(idx)
        # 读取同一个excel文件中的不同sheet
        print("Sheet", idx + 1)
        param_type = sheet.cell(10, 1).value
        if param_type == 'pressure':
            param_type = 'Leq'
        elif param_type == 'fluctuation strength':
            param_type = 'Fluct'
        else:
            # 不用小写
            # param_type = param_type.lower()
            # 首字母大写
            param_type = param_type.capitalize()
        values = []
        times = []

        for i in range(15, sheet.nrows):
            time = float(sheet.cell(i, 0).value)
            values.append(float(sheet.cell(i, 1).value))
            times.append(time)

        values = np.array(values)
        times = np.array(times)
        mean = values.mean()
        var = values.var()
        std = values.std(ddof=1)
        max = values.max()
        p_10 = np.percentile(values, 10, interpolation='midpoint')
        p_25 = np.percentile(values, 25, interpolation='midpoint')
        p_50 = np.percentile(values, 50, interpolation='midpoint')
        p_75 = np.percentile(values, 75, interpolation='midpoint')
        p_90 = np.percentile(values, 90, interpolation='midpoint')
        p_10_90 = p_90 - p_10

        params_dict["%s_mean" % param_type] = mean
        params_dict["%s_var" % param_type] = var
        params_dict["%s_std" % param_type] = std
        params_dict["%s_max" % param_type] = max
        params_dict["%s_10" % param_type] = p_10
        params_dict["%s_25" % param_type] = p_25
        params_dict["%s_median" % param_type] = p_50
        params_dict["%s_75" % param_type] = p_75
        params_dict["%s_90" % param_type] = p_90
        params_dict["%s_10_90" % param_type] = p_10_90

    soundtype_params = None
    try:
        file_name = os.path.basename(path).replace("xlsx", "wav")
        print(file_name, "is processed !")
        leq_mean = params_dict["Leq_mean"]
        leq_std = params_dict["Leq_std"]
        leq_25 = params_dict["Leq_25"]
        leq_median = params_dict["Leq_median"]
        leq_75 = params_dict["Leq_75"]
        leq_10_90 = params_dict["Leq_10_90"]
        loudness_mean = params_dict["Loudness_mean"]
        loudness_std = params_dict["Loudness_std"]
        loudness_25 = params_dict["Loudness_25"]
        loudness_median = params_dict["Loudness_median"]
        loudness_75 = params_dict["Loudness_75"]
        loudness_10_90 = params_dict["Loudness_10_90"]
        roughness_mean = params_dict["Roughness_mean"]
        roughness_std = params_dict["Roughness_std"]
        roughness_25 = params_dict["Roughness_25"]
        roughness_median = params_dict["Roughness_median"]
        roughness_75 = params_dict["Roughness_75"]
        roughness_10_90 = params_dict["Roughness_10_90"]
        sharpness_mean = params_dict["Sharpness_mean"]
        sharpness_std = params_dict["Sharpness_std"]
        sharpness_25 = params_dict["Sharpness_25"]
        sharpness_median = params_dict["Sharpness_median"]
        sharpness_75 = params_dict["Sharpness_75"]
        sharpness_10_90 = params_dict["Sharpness_10_90"]
        fluct_mean = params_dict["Fluct_mean"]
        fluct_std = params_dict["Fluct_std"]
        fluct_25 = params_dict["Fluct_25"]
        fluct_median = params_dict["Fluct_median"]
        fluct_75 = params_dict["Fluct_75"]
        fluct_10_90 = params_dict["Fluct_10_90"]
        tonality_mean = params_dict["Tonality_mean"]
        tonality_std = params_dict["Tonality_std"]
        tonality_25 = params_dict["Tonality_25"]
        tonality_median = params_dict["Tonality_median"]
        tonality_75 = params_dict["Tonality_75"]
        tonality_10_90 = params_dict["Tonality_10_90"]
        soundtype_params = SoundtypeParams(file_name, leq_mean, leq_std, leq_25, leq_median, leq_75, leq_10_90,
                                           loudness_mean, loudness_std, loudness_25, loudness_median, loudness_75,
                                           loudness_10_90, roughness_mean, roughness_std, roughness_25,
                                           roughness_median, roughness_75, roughness_10_90, sharpness_mean,
                                           sharpness_std, sharpness_25, sharpness_median, sharpness_75, sharpness_10_90,
                                           fluct_mean, fluct_std, fluct_25, fluct_median, fluct_75, fluct_10_90,
                                           tonality_mean, tonality_std, tonality_25, tonality_median, tonality_75,
                                           tonality_10_90)
    except Exception as e:
        print("[ERROR] XlsProcessor: %s" % e)

    params_dict["name"] = os.path.basename(path)
    single_data = pd.DataFrame(params_dict, [0])

    return single_data

In [58]:
# 测试例子
path = r'F:\中建科技项目数据\history\use\2021-12-27\xls\Record109848.xls'
example = extract_features(path)
columns = example.columns

Sheet 1
Sheet 2
Sheet 3
Sheet 4
Sheet 5
Sheet 6
Record109848.xls is processed !


In [74]:
# 新建一个dataframe用来存储麒麟山庄的特征
qilin_features = pd.DataFrame(columns=columns)
root_dir = r"F:\中建科技项目数据\history\use"

# 循环执行,只用qilin_use_data
for i in range(len(qilin_use_data)):
    file_name = qilin_use_data.loc[i,"file_name"] + ".xls"
    # 查找路径
    path = seek_files(root_dir, file_name)
    # 提取特征
    single_data = extract_features(path)
    # 记住要 = ！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！
    # 血泪史
    qilin_features = qilin_features.append(single_data,ignore_index=True)

qilin_features

Sheet 1
Sheet 2
Sheet 3
Sheet 4
Sheet 5
Sheet 6
Record85823.xls is processed !
Sheet 1
Sheet 2
Sheet 3
Sheet 4
Sheet 5
Sheet 6
Record85826.xls is processed !
Sheet 1
Sheet 2
Sheet 3
Sheet 4
Sheet 5
Sheet 6
Record85828.xls is processed !
Sheet 1
Sheet 2
Sheet 3
Sheet 4
Sheet 5
Sheet 6
Record85897.xls is processed !
Sheet 1
Sheet 2
Sheet 3
Sheet 4
Sheet 5
Sheet 6
Record85955.xls is processed !
Sheet 1
Sheet 2
Sheet 3
Sheet 4
Sheet 5
Sheet 6
Record85974.xls is processed !
Sheet 1
Sheet 2
Sheet 3
Sheet 4
Sheet 5
Sheet 6
Record85975.xls is processed !
Sheet 1
Sheet 2
Sheet 3
Sheet 4
Sheet 5
Sheet 6
Record85999.xls is processed !
Sheet 1
Sheet 2
Sheet 3
Sheet 4
Sheet 5
Sheet 6
Record86009.xls is processed !
Sheet 1
Sheet 2
Sheet 3
Sheet 4
Sheet 5
Sheet 6
Record86013.xls is processed !
Sheet 1
Sheet 2
Sheet 3
Sheet 4
Sheet 5
Sheet 6
Record86055.xls is processed !
Sheet 1
Sheet 2
Sheet 3
Sheet 4
Sheet 5
Sheet 6
Record86063.xls is processed !
Sheet 1
Sheet 2
Sheet 3
Sheet 4
Sheet 5
Sheet 6
Reco

,Leq_mean,Leq_var,Leq_std,Leq_max,Leq_10,Leq_25,Leq_median,Leq_75,Leq_90,Leq_10_90,...,Tonality_var,Tonality_std,Tonality_max,Tonality_10,Tonality_25,Tonality_median,Tonality_75,Tonality_90,Tonality_10_90,name
0,54.312528,53.647026,7.324739,69.625366,44.742218,46.711838,55.229630,60.548046,64.009647,19.267429,...,0.003439,0.058662,0.291835,0.0,0.019539,0.063939,0.109481,0.149417,0.149417,Record85823.xls
1,55.232130,28.291445,5.319207,69.071991,47.749233,50.331184,56.141373,59.358688,62.048265,14.299032,...,0.003874,0.062262,0.338405,0.0,0.046919,0.084511,0.128971,0.176758,0.176758,Record85826.xls
2,50.533419,16.909742,4.112328,61.366192,44.896023,47.537155,50.347477,53.674778,55.721466,10.825443,...,0.004035,0.063543,0.352343,0.0,0.028660,0.067247,0.114759,0.162089,0.162089,Record85828.xls
3,50.206056,10.631503,3.260744,62.288425,46.105684,47.749138,49.969536,52.371662,54.487446,8.381762,...,0.002846,0.053369,0.345522,0.0,0.010537,0.050323,0.090587,0.129303,0.129303,Record85897.xls
4,50.892217,29.397366,5.422175,62.937305,42.569418,46.039944,52.623150,55.334427,57.264042,14.694624,...,0.006525,0.080808,0.520856,0.0,0.035899,0.081288,0.134716,0.191422,0.191422,Record85955.xls
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,50.024887,0.921990,0.960246,60.606106,49.306311,49.514557,49.829697,50.255989,50.816889,1.510578,...,0.000993,0.031517,0.163329,0.0,0.000000,0.027650,0.051968,0.072974,0.072974,Record109726.xls
752,57.370135,26.701392,5.167569,67.176895,48.360319,54.507286,58.689892,61.229809,62.964729,14.604410,...,0.004562,0.067565,0.336235,0.0,0.042409,0.089499,0.137646,0.188131,0.188131,Record109758.xls
753,51.327230,3.189279,1.785935,61.432575,49.293150,49.961189,51.132454,52.213837,53.257622,3.964472,...,0.002586,0.050873,0.264383,0.0,0.022004,0.057294,0.093313,0.130287,0.130287,Record109784.xls
754,55.527740,40.705632,6.380380,69.133560,47.716015,48.599403,56.505791,61.127293,63.526405,15.810390,...,0.002881,0.053694,0.296857,0.0,0.027664,0.058366,0.097927,0.139450,0.139450,Record109802.xls


In [73]:
# test
single_data = extract_features(r'F:\中建科技项目数据\history\use\2021-12-27\xls\Record109848.xls')
single_data
a = pd.DataFrame(columns=columns)
a = a.append(single_data)
a = a.append(single_data)
a = a.append(single_data,ignore_index=True)
a

Sheet 1
Sheet 2
Sheet 3
Sheet 4
Sheet 5
Sheet 6
Record109848.xls is processed !


,Leq_mean,Leq_var,Leq_std,Leq_max,Leq_10,Leq_25,Leq_median,Leq_75,Leq_90,Leq_10_90,...,Tonality_var,Tonality_std,Tonality_max,Tonality_10,Tonality_25,Tonality_median,Tonality_75,Tonality_90,Tonality_10_90,name
0,45.775012,0.443905,0.666292,50.219433,45.046093,45.322029,45.708633,46.080547,46.518415,1.472322,...,0.001664,0.040808,0.216712,0.0,0.0,0.038961,0.068247,0.095698,0.095698,Record109848.xls
1,45.775012,0.443905,0.666292,50.219433,45.046093,45.322029,45.708633,46.080547,46.518415,1.472322,...,0.001664,0.040808,0.216712,0.0,0.0,0.038961,0.068247,0.095698,0.095698,Record109848.xls
2,45.775012,0.443905,0.666292,50.219433,45.046093,45.322029,45.708633,46.080547,46.518415,1.472322,...,0.001664,0.040808,0.216712,0.0,0.0,0.038961,0.068247,0.095698,0.095698,Record109848.xls


In [76]:
write_path = r'.\output\qilin_features.csv'
qilin_features.to_csv(write_path)